In [35]:
import torch
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_
import jieba
from tqdm import tqdm

# 定义超参数
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 100
batch_size = 50
seq_length = 30
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Dictionary(object):

    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def __len__(self):
        return len(self.word2idx)

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
            
# 定义语料库类
class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, batch_size=20):
        tokens = []
        lines = []

        with open(path, 'r', encoding="ANSI") as f:
            lines = f.readlines()

        for line in lines:
            words = jieba.lcut(line) + ['<eos>']
            tokens.extend(words)
            for word in words:
                self.dictionary.add_word(word)

        ids = torch.LongTensor([self.dictionary.word2idx[word] for word in tokens])
        num_batches = len(ids) // batch_size
        ids = ids[:num_batches * batch_size].view(batch_size, -1)

        return ids

# 定义LSTM模型类
class LSTMmodel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(LSTMmodel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        x = self.embed(x)
        out, (h, c) = self.lstm(x, h)
        out = out.reshape(out.size(0) * out.size(1), out.size(2))
        out = self.linear(out)

        return out, (h, c)

# 创建语料库实例并获取数据
corpus = Corpus()
ids = corpus.get_data('F:\\大学\\圆桌面\\深度学习与自然语言处理\\第三次作业\\jyxstxtqj_downcc.com\\鸳鸯刀.txt', batch_size)

# 获取词汇表大小
vocab_size = len(corpus.dictionary)

# 创建模型实例并将其移动到指定设备上
model = LSTMmodel(vocab_size, embed_size, hidden_size, num_layers).to(device)

# 定义损失函数和优化器
cost = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
progress_bar = tqdm(total=len(range(0, ids.size(1) - seq_length, seq_length)) * num_epochs, desc='Training')

for epoch in range(num_epochs):
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
              torch.zeros(num_layers, batch_size, hidden_size).to(device))

    for i in range(0, ids.size(1) - seq_length, seq_length):
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        states = [state.detach() for state in states]
        outputs, states = model(inputs, states)
        loss = cost(outputs, targets.reshape(-1))

        model.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        progress_bar.update(1)  # 手动更新进度条位置

progress_bar.close()  # 关闭进度条

# 生成文本
num_samples = 1000
article = ""

state = (torch.zeros(num_layers, 1, hidden_size).to(device),
        torch.zeros(num_layers, 1, hidden_size).to(device))

prob = torch.ones(vocab_size)
_input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)

for i in range(num_samples):
    output, state = model(_input, state)

    prob = output.exp()
    word_id = torch.multinomial(prob, num_samples=1).item()

    _input.fill_(word_id)

    word = corpus.dictionary.idx2word[word_id]
    word = '\n' if word == '<eos>' else word
    article += word

print(article)


Training: 100%|████████████████████████████████████████████████████████████████████| 1800/1800 [05:20<00:00,  5.62it/s]


讲什尽不论，江湖上人称八步赶蟾、赛专诸

　　、踏雪无痕、独脚水上飞、双刺盖七省。太岳四侠中排名第四。」那书生拱手道：「久

　　仰，久仰。」向花剑影道：「这一位仁兄呢？」

　　花剑影眉头一皱，道：「谁有空和你这酸丁称兄道弟？」一把推开那书僮，提起他

　　所挑的篮子一掂，入手只觉重甸甸的，心头一喜，打开篮子一看，不由得到抽一口凉气

　　，原来满篮子都是旧书。常长风喝道：「呸！都是废物。」那书生忙道：「仁兄此言差

　　矣！圣贤之书，如何能说是废物？有道是书中自有黄金屋。」常长风道：「书中有黄金

　　？这些破书一文钱一斤，有没人要。」这时盖一鸣以打开扁担头另一端的行李，除了布

　　被布衣之外，竟无丝毫值钱之物。太岳四侠都是好生失望。

　　那书生道：「在下游学寻母，得见四位仁兄，幸如何之？四位号称太岳四侠，想必是束手待毙

　　。你装腔作势，瞒得了别人，可乘早别在卓天雄眼前现世。」说著双刀平平一击，铮的

　　一响，声振林梢。

　　袁冠南右手提著一枝毛笔，左手平持一只墨盒，说道：「在下诗兴忽双鸣凤，意欲

　　来铁棒击地，我走到。」这时伸手到房中，关上了「卓天雄手的刀。」

　　只听林任二人不住口地吆喝招数。一个道：「喜结丝罗在乔木。」一个道：「英雄

　　无双风流婿。」一个道：「却扇洞房燃花烛。」一个道：「碧箫声里双鸣凤。」一个道

　　：「今朝有女颜如玉。」林玉龙叫道：「千金一刻庆良宵。」任飞燕叫道：「占断人间

　　天上福。」

　　喝到这里，那夫妻刀法的十二招以然使完，馀下尚有六十招，袁萧二人却未学过。

　　袁冠南叫道：「从头再来！」一刀破书出，便欲反唇相讥，口还未

　　，任飞燕也已割开了你的穴道。

　　~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

　　太岳四侠躲在密林之中，眼见威信镖局一行人走得远了，这才出来。花剑影撕下一

　　块衣襟，给逍遥子裹扎肩头的刀伤。常长风道：「大哥，不拌嘴不动刀子，否则便又是在府的英雄好汉？二人？

　　」另一人跟著赶到，喝道：「你有暗青子，我便没有麽？」拉开弹弓，吧吧吧一阵响，

　　八九枚连珠弹打了过来，有两枚打在马臀上，那马吃痛，後腿乱跳，登时将周威

In [43]:
import jieba
article="六粒盗，便问韦生道：“请你表演这项绝技。”又问左右道：“不用害怕，这件事与官人无关，不会累到你的。”说着提起皮囊，跃墙而出，体态轻盈，有一对夫妇。到后院，来到海边，戒备甚严，势似飞腾，寂无行迹，此必侠士，容貌。你性命，以匕首，大悦，收其策而退。当公之骋辩也，一妓有殊色，执红拂，立于前，独目公。公既去，而执拂者临轩，指吏曰：“问去者处士第几？，主人识得曰：“合负仆射万死。”这段文字既豪迈而又缠绵，有英雄之气，将他关了。王继恩说：“潘??公，主人西，便派你。”指着一名婢女道：“在下在市上有一间先父留下来的小店，每日可赚一贯，将复离叛。请郎君出，以金合授之。李郎明发，何日到太原？”靖计之曰：“主人西，则酒肆也。”公取到来，与姜廉夫同居如初。女剑仙水性杨花，男剑仙争风吃醋，都不成话。所以任渭长的评话说：“髑髅尽痴，剑仙如斯！”说着一名亲戚中，也知道了丈夫之意。那婢女当是她的心腹，她要丈夫一并杀了，以免道：“走罢！”董国庆又惊又喜，入房等妾同行。妾道：“我眼前有事，还不能走，明年"
# 分词处理
word_list = jieba.lcut(article)
unique_words = set(word_list)  # 获取唯一单词
num_unique_words = len(unique_words)  # 统计唯一单词的数量
word_ratio = num_unique_words / num_samples  # 计算唯一单词的比例

print("词汇多样性评估：")
print("唯一单词数量:", num_unique_words)
print("词汇丰富性比例:", word_ratio)

# 统计重复率
repeated_phrases = 0
total_phrases = len(word_list) - 1

for i in range(total_phrases):
    if word_list[i] == word_list[i+1]:
        repeated_phrases += 1

repetition_ratio = repeated_phrases / total_phrases  # 计算重复率

print("重复率评估：")
print("重复词组数量:", repeated_phrases)
print("重复率:", repetition_ratio)


词汇多样性评估：
唯一单词数量: 173
词汇丰富性比例: 0.173
重复率评估：
重复词组数量: 1
重复率: 0.0033444816053511705


In [41]:
import torch
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_
import jieba
from tqdm import tqdm

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}  # 单词到索引的映射
        self.idx2word = {}  # 索引到单词的映射
        self.idx = 0  # 索引计数器

    def __len__(self):
        return len(self.word2idx)

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.idx  # 将单词映射到索引
            self.idx2word[self.idx] = word  # 将索引映射到单词
            self.idx += 1  # 索引计数器加一
            
class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, batch_size=20):
        tokens = []  # 存储所有的单词
        lines = []  # 存储所有的行
        stopwords = [' ','wwwcrcom','www','cr173','com','□','说道','说','道','便','笑','=','txt','\n','本书来自www.cr173.com免费txt小说下载站','更多更新免费电子书请关注www.cr173.com','\u3000','目录']  # 定义广告
        with open(path, 'r', encoding="ANSI") as f:
            lines = f.readlines()  # 读取文件中的所有行

        for line in lines:
            words = jieba.lcut(line) + ['<eos>']  # 使用结巴分词对每一行进行分词，并在末尾添加结束标记
            words = [i for i in words if i not in stopwords] # 去除停用词，广告
            tokens.extend(words)  # 将分词结果添加到tokens列表中
            for word in words:
                self.dictionary.add_word(word)  # 将分词结果添加到词典中

        ids = torch.LongTensor([self.dictionary.word2idx[word] for word in tokens])  # 将单词转换为对应的索引
        num_batches = len(ids) // batch_size
        ids = ids[:num_batches * batch_size].view(batch_size, -1)  # 调整ids的形状为(batch_size, -1)

        return ids  

    
class LSTMmodel(nn.Module):

    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(LSTMmodel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        x = self.embed(x)
        out, (h, c) = self.lstm(x, h)
        out = out.reshape(out.size(0) * out.size(1), out.size(2))
        out = self.linear(out)

        return out, (h, c)

In [2]:
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 20
batch_size = 50
seq_length = 30
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
corpus = Corpus()
ids = corpus.get_data('F:\\大学\\圆桌面\\深度学习与自然语言处理\\第三次作业\\jyxstxtqj_downcc.com\\碧血剑.txt', batch_size)
vocab_size = len(corpus.dictionary)
model = LSTMmodel(vocab_size, embed_size, hidden_size, num_layers).to(device)
cost = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HUAWEI\AppData\Local\Temp\jieba.cache
Loading model cost 0.650 seconds.
Prefix dict has been built successfully.


In [4]:
progress_bar = tqdm(total=len(range(0, ids.size(1) - seq_length, seq_length)) * num_epochs, desc='Training')

for epoch in range(num_epochs):
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
              torch.zeros(num_layers, batch_size, hidden_size).to(device))

    for i in range(0, ids.size(1) - seq_length, seq_length):
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        states = [state.detach() for state in states]
        outputs, states = model(inputs, states)
        loss = cost(outputs, targets.reshape(-1))

        model.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        progress_bar.update(1)  # 手动更新进度条位置

progress_bar.close()  # 关闭进度条

Training: 100%|████████████████████████████████████████████████████████████████████| 4120/4120 [40:41<00:00,  1.69it/s]


In [5]:
num_samples = 400

article = str()

state = (torch.zeros(num_layers, 1, hidden_size).to(device),
        torch.zeros(num_layers, 1, hidden_size).to(device))

prob = torch.ones(vocab_size)
_input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)

In [7]:
for i in range(num_samples):
    output, state = model(_input, state)

    prob = output.exp()
    word_id = torch.multinomial(prob, num_samples=1).item()

    _input.fill_(word_id)

    word = corpus.dictionary.idx2word[word_id]
    word = '\n' if word == '<eos>' else word
    article += word
print(article)

清楚得到时？眼见越越越急叫了起来：“你出来，早就有人起你啦！”青青听他得不错，知道他是自己的。众人一路上对这个回答，：“先父当年镇守关辽，但还能不能再还我们的小做一笔了。”群豪神色同行，将那妇人送了几个头，直没至大的极大，三人吃了下去。仙都派围攻。
袁承志：“这是黄金的姑娘，都还了绿林中的。姓袁的却只有跟仙都派去的，是不是？”沙老大：“莲子一点不必，不知是哪一位？”
众人见过他身子，随手大奇，心想这人实在是锦衣毒丐，不由得大吃一惊，不知如何是好，才慢慢伸手出来，当即回到一两个酒杯，打得华山派华山派的精要。谢尚政对袁承志：“你陪我下棋，她是甚么大师？师叔可有二十年郑起云的？出来，你想救你的？”温仪低声：“我们是云南人吗？又来问他。那总不能有了事，想到是要来给我的？”那店小二听得温青叫他“小慧，别跟他会面。他马公子却是剑的较量呢？还是他真的叫他再在他上的五十两，他妈妈我温氏了，原来是他交出的人。”到这里，没又杀了一个徒弟，对袁承志：“我也这般答应了。怎么他死了，叫他就是甚么着实？”安小慧一拉，将他的手，放在仙都派去找他的五行拳。原来他东和少女都吃了一惊，终于吃了一惊，这时更加二更紧，在火光口，只听得她有人避开，心头手，举起锅盖向他肩头打去。袁承志眼见她转瞬之间，要死却也知人家的仙都门徒焦公礼江南不如安小慧。只听他高声：“好，你也来问我。”梅剑和一愣，向他脸上做手势而去。青青到陪得瞧着钢套，十分假意，轻轻揭开单刀，：“你打他是甚么人。怎么我们有坐在这里。”那瘦子：“是啊！”原来了那是厉害的站了起来，心想：“难道这剑竟是五毒教的人，跟他们学武于他的朋友。”她想到袁承志心中恼怒，：“不错，正是他。袁承志除了曹公公磕头，当下两位只有都碰了。”向她：“我们要怎样，你也一起知道么？”袁承志：“大师哥的枪法，都知道闯王的了。”李岩不知这批更高，又想：“他以决不能跟他会面。他若是五兄弟，他老人家有怕我能能？”安大娘：“我兄弟失手伤你的师哥，只怕要知道你才他还待现今人也都不错。”马公子大喜，：“好呀，那么你这把匕首，就饶了这娃儿。”猛喝：“我也问这个呢，还是叫做他的送命多，让你还记得清清楚楚？哈哈，忙伸手起身让她父皇的客气……”焦公礼叹：“倘若祖叔叔害他的，还是阿九的兄弟找他说话话，但这两个女贼之事不知却是难说。他早知我也曾和他下毒。他手法之后，全身右掌，忙问他把三分和他们图谋呢？”冯不

In [13]:
import torch
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_
import jieba
from tqdm import tqdm

# 定义超参数
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 10
batch_size = 50
seq_length = 30
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 定义语料库类
class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, batch_size=20):
        tokens = []
        lines = []

        with open(path, 'r', encoding="ANSI") as f:
            lines = f.readlines()

        for line in lines:
            words = jieba.lcut(line) + ['<eos>']
            tokens.extend(words)
            for word in words:
                self.dictionary.add_word(word)

        ids = torch.LongTensor([self.dictionary.word2idx[word] for word in tokens])
        num_batches = len(ids) // batch_size
        ids = ids[:num_batches * batch_size].view(batch_size, -1)

        return ids

# 定义LSTM模型类
class LSTMmodel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(LSTMmodel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        x = self.embed(x)
        out, (h, c) = self.lstm(x, h)
        out = out.reshape(out.size(0) * out.size(1), out.size(2))
        out = self.linear(out)

        return out, (h, c)

# 创建语料库实例并获取数据
corpus = Corpus()
ids = corpus.get_data('C:\\Users\\HUAWEI\\Desktop\\深度学习与自然语言处理\\第三次作业\\jyxstxtqj_downcc.com\\雪山飞狐.txt', batch_size)

# 获取词汇表大小
vocab_size = len(corpus.dictionary)

# 创建模型实例并将其移动到指定设备上
model = LSTMmodel(vocab_size, embed_size, hidden_size, num_layers).to(device)

# 定义损失函数和优化器
cost = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
for epoch in range(num_epochs):
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
              torch.zeros(num_layers, batch_size, hidden_size).to(device))

    for i in tqdm(range(0, ids.size(1) - seq_length, seq_length)):
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        states = [state.detach() for state in states]
        outputs, states = model(inputs, states)
        loss = cost(outputs, targets.reshape(-1))

        model.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

# 描述训练过程
print("训练完成！模型已经学习了语料库中的模式和规律。")


100%|██████████████████████████████████████████████████████████████████████████████████| 61/61 [01:54<00:00,  1.88s/it]

训练完成！模型已经学习了语料库中的模式和规律。


In [16]:
num_samples = 600

article = str()

state = (torch.zeros(num_layers, 1, hidden_size).to(device),
        torch.zeros(num_layers, 1, hidden_size).to(device))

prob = torch.ones(vocab_size)
_input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)
for i in range(num_samples):
    output, state = model(_input, state)

    prob = output.exp()
    word_id = torch.multinomial(prob, num_samples=1).item()

    _input.fill_(word_id)

    word = corpus.dictionary.idx2word[word_id]
    word = '\n' if word == '<eos>' else word
    article += word
print(article)

转脚於惨叫』今日酒来，满清话。范田三人到捏停住，拨一步，初时试试旁跃开，另外他提剑将湖斐放在的不，要在下无耻军中了威名，明明偷鸡摸狗、刺。只见一格他神情，那人是自尽，在下见到也叫道：『胡一刀？那一边明天说完劳。那知那阎基英雄剑法，覆以，就跟您老。』胡一刀道：『怎麽军令？』」于管家道：「跟各位要哥哥，外边兵刃响当当？」众人问上前烛火，都难去拿。

　　陶百岁这 叫道：「你有种英雄无敌了，我跟著跃下。』只见只听出来手臂，坐出神。当晚、总管羊肉逝世名山，不敢一日，显得也未能。金面佛胡一刀交替房外，下官就欢喜。唉了。」宝树半步一声，无不一听碰到。

　　胡斐听到道：「倘若，你此间这样，那麽凭还谁？」平阿四道：「若是你懊恼三朝了这一点点大豪跟我姓。你若有拿！』，爹爹一到，不进迫。静智吓了田归农意思。」

　　田青文道：「自是走，我自幼知道。两位小说后，手下夫人与他怎生九宫山，功夫奉，以及砍仇。姑娘主人於，也昏有什麽？」

　　　「那夫人叱，见刀鞘寻宝，两人进退温暖，拍的大嘴的奇一阵响，[门框两步，提起酒壶他的肌肉了。这大气早的逃生，在下百摺裙摔得。共有。来拿在半夜，急忙的僮儿、嘿、祖宗一奇，从里身上上一片雪峰，只听鼻肿，又往嘱咐中论九宫山几十斤。脚夫公公轻响，拳脚发表，这拂不会人。那老僧纵上，一个实在一沉，片刻间应变中的物事，愈来愈是武林。但见他敌手也是南北的情景，数次猛扑也与闻。那三人见敌剑、阮士中等都为踢。陶百岁二人甚是一出手能服，原来是生晕，但有所而起，那老僧心中都拨貂裘。殷吉是背向译本。

　　他抛落起一脚，一摸小人），只瞧陶子安恋战，向前紧紧裹住在地，必定之间，原书甚麽久了师兄。劲力抓住退他手里赶之外，原本钢鞭众人点燃。

　　胡斐宝树喝道：「我爹远来剑蝗敌人陷害，深者重报，形似尽有掉，只是身子、陶百岁、平通镖局、馒头、昆仑山他并肩而行，更是险象环生。两人翻翻滚滚总算，生怕在他与陶百岁的眼光本门好人。她当时听著扶胡相公。苗姑娘第三招上，苗大侠去睡觉，起初她身子就是了。柄於的偷鸡摸狗，三枚那时，拾起下。他动起一挺，然而不知三家对外的刀法倒
